In [1]:
!pip install category_encoders -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 637.8 kB/s eta 0:00:00


In [2]:
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import category_encoders as ce

from sklearn.metrics import mean_absolute_error, mean_squared_error

# KNN для задачи регрессии

https://ggplot2.tidyverse.org/reference/diamonds.html

In [3]:
diamonds = sns.load_dataset('diamonds')

In [4]:
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


## EDA

In [5]:
diamonds.describe(include='all')

,carat,cut,color,clarity,depth,table,price,x,y,z
count,53940.000000,53940,53940,53940,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
unique,NaN,5,7,8,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Ideal,G,SI1,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,21551,11292,13065,NaN,NaN,NaN,NaN,NaN,NaN
mean,0.797940,NaN,NaN,NaN,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,NaN,NaN,NaN,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,NaN,NaN,NaN,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,NaN,NaN,NaN,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,NaN,NaN,NaN,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,NaN,NaN,NaN,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000


In [6]:
diamonds.isna().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64

In [7]:
diamonds['color'].value_counts()

color
G    11292
E     9797
F     9542
H     8304
D     6775
I     5422
J     2808
Name: count, dtype: int64

In [8]:
diamonds['clarity'].value_counts()

clarity
SI1     13065
VS2     12258
SI2      9194
VS1      8171
VVS2     5066
VVS1     3655
IF       1790
I1        741
Name: count, dtype: int64

In [9]:
diamonds['cut'].value_counts()

cut
Ideal        21551
Premium      13791
Very Good    12082
Good          4906
Fair          1610
Name: count, dtype: int64

In [10]:
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


## разделим на трейн и тест

In [11]:
X_reg = diamonds.drop(columns=['price'])
y_reg = diamonds['price']

In [12]:
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)


In [13]:
X_train_reg.shape, X_test_reg.shape

((43152, 9), (10788, 9))

In [14]:
X_train_reg = X_train_reg.reset_index().drop(columns='index')
X_test_reg = X_test_reg.reset_index().drop(columns='index')

In [15]:
y_train_reg = y_train_reg.reset_index().drop(columns='index')
y_test_reg = y_test_reg.reset_index().drop(columns='index')

In [16]:
X_train_reg.shape, X_test_reg.shape

((43152, 9), (10788, 9))

In [17]:
features_number = X_train_reg.select_dtypes(include='number').columns

In [18]:
features_cat = X_train_reg.select_dtypes(exclude='number').columns

## подготовка датасета

### StandardScaler

In [19]:
scaler = StandardScaler()

In [20]:
X_train_reg_number = pd.DataFrame(scaler.fit_transform(X_train_reg[features_number]), columns=features_number)
X_test_reg_number = pd.DataFrame(scaler.transform(X_test_reg[features_number]), columns=features_number)


In [21]:
X_train_reg_number.shape, X_test_reg_number.shape

((43152, 6), (10788, 6))

In [22]:
X_train_reg = X_train_reg_number.merge(X_train_reg[features_cat], left_index=True, right_index=True)
X_train_reg.shape

(43152, 9)

In [23]:
X_test_reg = X_test_reg_number.merge(X_test_reg[features_cat], left_index=True, right_index=True)
X_test_reg.shape

(10788, 9)

In [24]:
X_train_reg.head()

,carat,depth,table,x,y,z,cut,color,clarity
0,2.560056,-2.550748,2.933861,2.229450,2.138209,1.738207,Good,F,SI2
1,0.447392,-1.220426,1.139575,0.747550,0.656710,0.537733,Very Good,E,SI2
2,0.637532,0.529996,0.242432,0.765404,0.700284,0.791951,Premium,H,VS2
3,1.482597,-0.170173,3.382432,1.318885,1.249310,1.243894,Good,E,SI2
4,1.524851,0.249928,-0.206140,1.372448,1.380030,1.399249,Very Good,G,VS1


In [25]:
X_test_reg.head()

,carat,depth,table,x,y,z,cut,color,clarity
0,-1.179359,0.249928,-0.654711,-1.573499,-1.513250,-1.510133,Ideal,G,VVS1
1,-0.461053,-1.220426,-0.206140,-0.261214,-0.275763,-0.394399,Very Good,F,VVS2
2,-0.841333,0.249928,-1.103283,-0.868257,-0.868362,-0.832219,Ideal,E,VVS2
3,-0.777953,-0.660291,-0.206140,-0.725423,-0.737642,-0.789849,Premium,E,VVS2
4,1.588231,0.389962,-1.103283,1.524209,1.423604,1.512235,Ideal,E,SI2


### Mean Target Encoding

In [26]:
# Mean target encoding для признака 'cut'
target_enc = ce.TargetEncoder(cols=['cut'])
X_train_reg['cut'] = target_enc.fit_transform(X_train_reg['cut'], y_train_reg)
X_test_reg['cut'] = target_enc.transform(X_test_reg['cut'])

In [27]:
# Mean target encoding для признака 'cut'
target_enc = ce.TargetEncoder(cols=['clarity'])
X_train_reg['clarity'] = target_enc.fit_transform(X_train_reg['clarity'], y_train_reg)
X_test_reg['clarity'] = target_enc.transform(X_test_reg['clarity'])

In [28]:
X_test_reg.head()

,carat,depth,table,x,y,z,cut,color,clarity
0,-1.179359,0.249928,-0.654711,-1.573499,-1.513250,-1.510133,3470.103019,G,2544.185802
1,-0.461053,-1.220426,-0.206140,-0.261214,-0.275763,-0.394399,3987.217938,F,3296.911227
2,-0.841333,0.249928,-1.103283,-0.868257,-0.868362,-0.832219,3470.103019,E,3296.911227
3,-0.777953,-0.660291,-0.206140,-0.725423,-0.737642,-0.789849,4568.368827,E,3296.911227
4,1.588231,0.389962,-1.103283,1.524209,1.423604,1.512235,3470.103019,E,5069.865453


In [29]:
X_train_reg.head()

,carat,depth,table,x,y,z,cut,color,clarity
0,2.560056,-2.550748,2.933861,2.229450,2.138209,1.738207,3965.815736,F,5069.865453
1,0.447392,-1.220426,1.139575,0.747550,0.656710,0.537733,3987.217938,E,5069.865453
2,0.637532,0.529996,0.242432,0.765404,0.700284,0.791951,4568.368827,H,3917.891862
3,1.482597,-0.170173,3.382432,1.318885,1.249310,1.243894,3965.815736,E,5069.865453
4,1.524851,0.249928,-0.206140,1.372448,1.380030,1.399249,3987.217938,G,3832.311332


In [30]:
X_train_reg.shape, X_test_reg.shape

((43152, 9), (10788, 9))

### OneHotEncoder

In [31]:
# One-hot кодирование признака 'cut'
X_test_reg = pd.get_dummies(X_test_reg, columns=['color'], drop_first=True)
X_train_reg = pd.get_dummies(X_train_reg, columns=['color'], drop_first=True)

In [32]:
X_train_reg.shape, X_test_reg.shape

((43152, 14), (10788, 14))

In [33]:
X_train_reg.head()

,carat,depth,table,x,y,z,cut,clarity,color_E,color_F,color_G,color_H,color_I,color_J
0,2.560056,-2.550748,2.933861,2.229450,2.138209,1.738207,3965.815736,5069.865453,False,True,False,False,False,False
1,0.447392,-1.220426,1.139575,0.747550,0.656710,0.537733,3987.217938,5069.865453,True,False,False,False,False,False
2,0.637532,0.529996,0.242432,0.765404,0.700284,0.791951,4568.368827,3917.891862,False,False,False,True,False,False
3,1.482597,-0.170173,3.382432,1.318885,1.249310,1.243894,3965.815736,5069.865453,True,False,False,False,False,False
4,1.524851,0.249928,-0.206140,1.372448,1.380030,1.399249,3987.217938,3832.311332,False,False,True,False,False,False


In [34]:
X_test_reg.head()

,carat,depth,table,x,y,z,cut,clarity,color_E,color_F,color_G,color_H,color_I,color_J
0,-1.179359,0.249928,-0.654711,-1.573499,-1.513250,-1.510133,3470.103019,2544.185802,False,False,True,False,False,False
1,-0.461053,-1.220426,-0.206140,-0.261214,-0.275763,-0.394399,3987.217938,3296.911227,False,True,False,False,False,False
2,-0.841333,0.249928,-1.103283,-0.868257,-0.868362,-0.832219,3470.103019,3296.911227,True,False,False,False,False,False
3,-0.777953,-0.660291,-0.206140,-0.725423,-0.737642,-0.789849,4568.368827,3296.911227,True,False,False,False,False,False
4,1.588231,0.389962,-1.103283,1.524209,1.423604,1.512235,3470.103019,5069.865453,True,False,False,False,False,False


In [35]:
X_train_reg.head()

,carat,depth,table,x,y,z,cut,clarity,color_E,color_F,color_G,color_H,color_I,color_J
0,2.560056,-2.550748,2.933861,2.229450,2.138209,1.738207,3965.815736,5069.865453,False,True,False,False,False,False
1,0.447392,-1.220426,1.139575,0.747550,0.656710,0.537733,3987.217938,5069.865453,True,False,False,False,False,False
2,0.637532,0.529996,0.242432,0.765404,0.700284,0.791951,4568.368827,3917.891862,False,False,False,True,False,False
3,1.482597,-0.170173,3.382432,1.318885,1.249310,1.243894,3965.815736,5069.865453,True,False,False,False,False,False
4,1.524851,0.249928,-0.206140,1.372448,1.380030,1.399249,3987.217938,3832.311332,False,False,True,False,False,False


## построим knn модель

In [36]:
param_grid = {'n_neighbors': range(1, 21, 5),
              'p': [1, 2]}
knn = KNeighborsRegressor()
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_reg, y_train_reg)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': range(1, 21, 5), 'p': [1, 2]},
             scoring='neg_mean_squared_error')

In [37]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_p,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.216929,0.031017,1.437130,0.554406,1,1,"{'n_neighbors': 1, 'p': 1}",-752172.533194,-802334.285367,-816353.616570,-805815.859444,-739964.867323,-783328.232380,31011.799183,6
1,0.284837,0.088704,1.262397,0.272341,1,2,"{'n_neighbors': 1, 'p': 2}",-940803.886108,-950334.106361,-930416.226419,-958610.773465,-878466.608343,-931726.320139,28247.969799,8
2,0.184641,0.023787,1.416063,0.319305,6,1,"{'n_neighbors': 6, 'p': 1}",-516185.844182,-598405.975228,-613081.941425,-576553.008459,-536833.962434,-568212.146346,36567.368886,1
3,0.108463,0.024797,0.559144,0.024223,6,2,"{'n_neighbors': 6, 'p': 2}",-623211.771010,-683685.714466,-745233.127408,-694769.102511,-675663.064082,-684512.555895,39061.407944,4
4,0.092995,0.000957,0.681773,0.020009,11,1,"{'n_neighbors': 11, 'p': 1}",-544364.187005,-656271.364228,-682619.295407,-617954.838263,-596929.540289,-619627.845038,47944.343835,2
5,0.095126,0.005579,0.675615,0.150341,11,2,"{'n_neighbors': 11, 'p': 2}",-676724.041052,-759631.911873,-816405.508830,-744192.743450,-723714.148899,-744133.670821,45660.557587,5
6,0.117665,0.030935,0.916065,0.271257,16,1,"{'n_neighbors': 16, 'p': 1}",-589591.915408,-725906.528150,-740930.922237,-660515.491868,-661441.338662,-675677.239265,54081.876885,3
7,0.093109,0.001592,0.630673,0.016802,16,2,"{'n_neighbors': 16, 'p': 2}",-723987.511239,-832507.003703,-887853.261353,-793482.772816,-790456.832332,-805657.476289,53900.483073,7


In [38]:
grid_search.best_params_

{'n_neighbors': 6, 'p': 1}

Двойная звездочка ** перед grid_search.best_params_ используется для распаковки словаря в качестве аргументов функции.

In [39]:
knn_best = KNeighborsRegressor(**grid_search.best_params_)

In [40]:
knn_best.fit(X_train_reg, y_train_reg)

KNeighborsRegressor(n_neighbors=6, p=1)

## посчитаем метрики

In [41]:
y_pred_train = knn_best.predict(X_train_reg)
y_pred_test = knn_best.predict(X_test_reg)

In [42]:

# Вычисление метрик
mae_train = mean_absolute_error(y_train_reg, y_pred_train)
rmse_train = mean_squared_error(y_train_reg, y_pred_train, squared=False)

# Вычисление метрик
mae_test = mean_absolute_error(y_test_reg, y_pred_test)
rmse_test = mean_squared_error(y_test_reg, y_pred_test, squared=False)

In [43]:
mae_train, mae_test

(308.06221032628844, 368.47376714868375)

In [44]:
rmse_train, rmse_test

(608.0462305547733, 727.7792534888583)

In [45]:
y_train_reg.mean().values[0], y_test_reg.mean().values[0]

(3939.490707267334, 3906.0357804968485)